In [10]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from MayaDataset import CNNCLF
import MayaDataset
from DefenderGAN import Warmup
from Models import RNNGenerator2, RNNInference
import argparse
import time
import os
victim = 'video'
dataset = MayaDataset.MayaDataset('raw_video',minpower=30, maxpower=200, window=1400, labels=victim)

dsets = random_split(dataset, [1000,1000,1000, 1000])
trainset = dsets[0]
trainloader = DataLoader(trainset, batch_size=32, num_workers=4)

valset = dsets[1]
valloader = DataLoader(valset, batch_size=32, num_workers=4)

testset = dsets[2]
testloader = DataLoader(testset, batch_size=32, num_workers=4)

dim=64
clf = CNNCLF(dataset.window).cuda()
gen = RNNGenerator2(dim).cuda()
#bestname = './best_{}_{}_{}.pth'.format(victim,'rnn2',dim)
bestname = './best_{}_{}.pth'.format('rnn2',dim)
if os.path.isfile(bestname):
    print('Previous best found: loading the model...')
    gen.load_state_dict(torch.load(bestname))
gen.train()


Previous best found: loading the model...


RNNGenerator2(
  (encoder): Sequential(
    (0): NoiseInjector()
    (1): Conv1d(1, 64, kernel_size=(32,), stride=(1,))
    (2): ReLU()
    (3): NoiseInjector()
  )
  (resblock): GRU(64, 64, batch_first=True)
  (decoder): Sequential(
    (0): NoiseInjector()
    (1): Linear(in_features=64, out_features=1, bias=True)
    (2): Hardtanh(min_val=-1.0, max_val=1.0)
  )
)

In [11]:
optim_c = torch.optim.Adam(clf.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
bestacc = 0.0
epochs = 30
prange = dataset.maxpower - dataset.minpower
pmin = dataset.minpower
for e in range(epochs):
    clf.train()
    for x,y in trainloader:
        optim_c.zero_grad()
        xdata, ydata = x.cuda(), y.cuda()
        perturb = gen(xdata)
        p_input = torch.relu(xdata + perturb.detach())
        output = clf(p_input)
        loss = criterion(output, ydata)
        loss.backward()
        optim_c.step()
    totcorrect = 0
    totcount = 0
    clf.eval()
    orgpower = 0.0
    newpower = 0.0
    for x,y in valloader:
        xdata, ydata = x.cuda(), y.cuda()
        orgpower += xdata.mean().item()
        perturb = gen(xdata)
        p_input = torch.relu(xdata + perturb.detach())
        newpower += p_input.mean().item()
        output = clf(p_input)
        pred = output.argmax(axis=-1)
        totcorrect += (pred==ydata).sum().item()
        totcount += y.size(0)
    macc = float(totcorrect)/totcount
    orgpower = orgpower/len(valloader)
    orgpower = orgpower*prange + pmin
    newpower = newpower/len(valloader)
    newpower = newpower*prange + pmin
    if(macc > bestacc) and e > epochs//2:
        bestacc = macc
        bestclf = clf.state_dict()
    print("Cooldown {}\t acc {:.4f}\torgpower {:.2f}\t newpower: {:.2f}".format(e+1, macc, orgpower, newpower))

clf.load_state_dict(bestclf)
totcorrect = 0
totcount = 0
clf.eval()
orgpower = 0.0
newpower = 0.0
for x,y in testloader:
    xdata, ydata = x.cuda(), y.cuda()
    orgpower += xdata.mean().item()
    perturb = gen(xdata)
    p_input = torch.relu(xdata + perturb.detach())
    newpower += p_input.mean().item()
    output = clf(p_input)
    pred = output.argmax(axis=-1)
    totcorrect += (pred==ydata).sum().item()
    totcount += y.size(0)
macc = float(totcorrect)/totcount
orgpower = orgpower/len(valloader)
orgpower = orgpower*prange + pmin
newpower = newpower/len(valloader)
newpower = newpower*prange + pmin
print("Test acc {:.4f}\torgpower {:.2f}\t newpower: {:.2f}".format(macc, orgpower, newpower))

Cooldown 1	 acc 0.2120	orgpower 79.28	 newpower: 65.84
Cooldown 2	 acc 0.3310	orgpower 79.28	 newpower: 65.84
Cooldown 3	 acc 0.6380	orgpower 79.28	 newpower: 65.84
Cooldown 4	 acc 0.6620	orgpower 79.28	 newpower: 65.85
Cooldown 5	 acc 0.6830	orgpower 79.28	 newpower: 65.85
Cooldown 6	 acc 0.6610	orgpower 79.28	 newpower: 65.84
Cooldown 7	 acc 0.6670	orgpower 79.28	 newpower: 65.84
Cooldown 8	 acc 0.7160	orgpower 79.28	 newpower: 65.85
Cooldown 9	 acc 0.6800	orgpower 79.28	 newpower: 65.84
Cooldown 10	 acc 0.6790	orgpower 79.28	 newpower: 65.85
Cooldown 11	 acc 0.7010	orgpower 79.28	 newpower: 65.84
Cooldown 12	 acc 0.7150	orgpower 79.28	 newpower: 65.84
Cooldown 13	 acc 0.7010	orgpower 79.28	 newpower: 65.84
Cooldown 14	 acc 0.7080	orgpower 79.28	 newpower: 65.84
Cooldown 15	 acc 0.7100	orgpower 79.28	 newpower: 65.84
Cooldown 16	 acc 0.6990	orgpower 79.28	 newpower: 65.85
Cooldown 17	 acc 0.7150	orgpower 79.28	 newpower: 65.84
Cooldown 18	 acc 0.6990	orgpower 79.28	 newpower: 65.84
C

In [6]:
clf.load_state_dict(bestclf)
totcorrect = 0
totcount = 0
clf.eval()
orgpower = 0.0
newpower = 0.0
for x,y in testloader:
    xdata, ydata = x.cuda(), y.cuda()
    orgpower += xdata.mean().item()
    perturb = gen(xdata)
    p_input = torch.relu(xdata + perturb.detach())
    newpower += p_input.mean().item()
    output = clf(p_input)
    pred = output.argmax(axis=-1)
    totcorrect += (pred==ydata).sum().item()
    totcount += y.size(0)
macc = float(totcorrect)/totcount
orgpower = orgpower/len(valloader)
orgpower = orgpower*prange + pmin
newpower = newpower/len(valloader)
newpower = newpower*prange + pmin
print("Test acc {:.4f}\torgpower {:.2f}\t newpower: {:.2f}".format(macc, orgpower, newpower))

Test acc 0.2550	orgpower 79.83	 newpower: 92.61


In [ ]:
train_x = []
train_y = []
valid_x = []
valid_y = []
for x,y in trainloader:
    xdata, ydata = x.cuda(), y.cuda()
    perturb = gen(xdata)
    p_input = torch.relu(xdata + perturb.detach())
    for p in p_input.cpu():
        train_x.append(p.numpy())
    for label in y:
        train_y.append(label)

for x,y in testloader:
    xdata, ydata = x.cuda(), y.cuda()
    perturb = gen(xdata)
    p_input = torch.relu(xdata + perturb.detach())
    for p in p_input.cpu():
        valid_x.append(p.numpy())
    for label in y:
        valid_y.append(label)
    

In [ ]:

from sklearn import svm
clf = svm.LinearSVC()
clf.fit(train_x, train_y)
pred_y = clf.predict(valid_x)

In [ ]:
correct = [yp == y for yp,y in zip(pred_y, valid_y)]
sum(correct)/len(correct)

In [ ]:
print(clf.coef_.shape)
print(clf.intercept_.shape)

In [8]:
model = RNNInference(dim)
model.copy_params(gen.cpu())
script_module = torch.jit.trace(model, (torch.rand(1,32), torch.rand(1,dim)))

/home/hwnam/anaconda3/envs/torch/lib/python3.7/site-packages/torch/jit/_trace.py:991: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1 / 1 (100.0%)
Greatest absolute difference: 0.19602189026772976 at index (0, 0) (up to 1e-05 allowed)
Greatest relative difference: 1.1124534356754818 at index (0, 0) (up to 1e-05 allowed)
  _module_class,
/home/hwnam/anaconda3/envs/torch/lib/python3.7/site-packages/torch/jit/_trace.py:991: TracerWarning: Output nr 2. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 64 / 64 (100.0%)
Greatest absolute difference: 0.08395768702030182 at index (0, 59) (up to 1e-05 allowed)
Greatest relative difference: 178.0198064169206 at index (0, 27) (up to 1e-05 allowed)
  _module_class,


In [9]:
script_module.save('./cpuscript_{}_{}_{}.pt'.format(victim,'rnn2',dim))

In [ ]:
aa = nn.Linear(32,64)
model.encoder[1].weight.shape
